In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from tabulate import tabulate

## User Input


In [ ]:
print("Do you want personalized planning by providing some information like Height, Weight, etc.?")
print("If YES, Press X = 1, Otherwise X = 0")
x = int(input("X = "))

## Calculating BMR and Nutrients Constraints

In [ ]:
# BMR
def calculate_bmr(gender, weight, height, age):
    if gender.lower() == 'male':
        return (10 * weight) + (6.25 * height) - (5 * age) + 5
    else:
        return (10 * weight) + (6.25 * height) - (5 * age) - 161

if x == 1:
    user = {}
    user['name'] = input("Enter your name: ")
    user['age'] = int(input("Enter your age: "))
    user['gender'] = input("Enter your gender (male/female): ").lower()
    user['height'] = float(input("Enter your height in cm: "))
    user['weight'] = float(input("Enter your weight in kg: "))

    bmr1 = calculate_bmr(user['gender'], user['weight'], user['height'], user['age'])
    print(f"Calculated BMR: {bmr1:.2f} kcal")

    min_cal = bmr1 * 1.2
    max_cal = bmr1 * 1.9
    bmr = bmr1 * 1.2
    Pconst = (bmr * 0.20) / 4
    Fconst = (bmr * 0.25) / 9
    Sconst = (bmr * 0.12) / 4
    Sodium_const = 2500
else:
    min_cal = 2200
    max_cal = 3500
    Pconst = 110
    Fconst = 61
    Sconst = 66
    Sodium_const = 2500

print(f"\n[Constraints]")
print(f"min_cal: {min_cal:.0f}, max_cal: {max_cal:.0f}, min Protein: {Pconst:.0f}, max_fat: {Fconst:.0f}, max_sugar: {Sconst:.0f}, max_sodium: {Sodium_const}")


## Load Dataset


In [ ]:
# 3. Load your dataset
data = pd.read_csv("../dataset/7_11_food_choices.csv")
data



## Data Extraction

In [ ]:
# Extracting the data and convert it to the list
prices = data['Price(baht)'].tolist()
calories = data['Calories(kCal)'].tolist()
proteins = data['Protein(g)'].tolist()
fats = data['Fat(g)'].tolist()
sugars = data['Sugar(g)'].tolist()
sodium = data['Sodium(mg)'].tolist()
items = data['ITEM'].tolist()

# Number of food items
n = len(items)
min_item = 3
max_item = 10


## NSGA - II

In [ ]:
#Problem class with added constraints
class FoodSelectionProblem(Problem):
    def __init__(self, data, min_item, max_item, min_cal, max_cal,
                 min_protein, max_fat, max_sugar, max_sodium):
        self.data = data
        self.n_items = len(data)
        self.min_item = min_item
        self.max_item = max_item
        self.min_cal = min_cal
        self.max_cal = max_cal
        self.min_protein = min_protein
        self.max_fat = max_fat
        self.max_sugar = max_sugar
        self.max_sodium = max_sodium

        super().__init__(n_var=self.n_items,
                         n_obj=2,
                         n_constr=8,
                         xl=0,
                         xu=1,
                         type_var=np.bool_)

    def _evaluate(self, X, out, *args, **kwargs):
        mask = X.astype(bool)

        total_cost = np.sum(data['Price(baht)'].values * mask, axis=1)
        total_protein = np.sum(data['Protein(g)'].values * mask, axis=1)

        total_calories = np.sum(data['Calories(kCal)'].values * mask, axis=1)
        total_fat = np.sum(data['Fat(g)'].values * mask, axis=1)
        total_sugar = np.sum(data['Sugar(g)'].values * mask, axis=1)
        total_sodium = np.sum(data['Sodium(mg)'].values * mask, axis=1)
        total_items = np.sum(mask, axis=1)

        g1 = self.min_item - total_items
        g2 = total_items - self.max_item
        g3 = self.min_cal - total_calories
        g4 = total_calories - self.max_cal
        g5 = self.min_protein - total_protein
        g6 = total_fat - self.max_fat
        g7 = total_sugar - self.max_sugar
        g8 = total_sodium - self.max_sodium

        out["F"] = np.column_stack([total_cost, -total_protein])  # Maximize protein → minimize -protein
        out["G"] = np.column_stack([g1, g2, g3, g4, g5, g6, g7, g8])

In [ ]:
# NSGA Execution across multiple runs
n_runs = 5
n_gen = 200
best_costs, avg_costs = [], []
best_proteins, avg_proteins = [], []

for run in range(n_runs):
    print(f"\n Running GA {run + 1}/{n_runs}")

    problem = FoodSelectionProblem(data,
                                   min_item=min_item,
                                   max_item=max_item,
                                   min_cal=min_cal,
                                   max_cal=max_cal,
                                   min_protein=Pconst,
                                   max_fat=Fconst,
                                   max_sugar=Sconst,
                                   max_sodium=Sodium_const)

    algorithm = NSGA2(
        pop_size=100,
        sampling=BinaryRandomSampling(),
        crossover=TwoPointCrossover(),
        mutation=BitflipMutation(),
        eliminate_duplicates=True
    )

    termination = get_termination("n_gen", n_gen)

    res = minimize(problem,
                   algorithm,
                   termination,
                   seed=run + 1,
                   save_history=True,
                   verbose=True)

    F = res.F
    best_costs.append(np.min(F[:, 0]))
    avg_costs.append(np.mean(F[:, 0]))
    best_proteins.append(-np.min(F[:, 1]))
    avg_proteins.append(-np.mean(F[:, 1]))
    
    best_cost = np.min(F[:, 0])
    best_protein = -np.min(F[:, 1])
    print(f"best_cost:{best_cost}")
    print(f"best_protein:{best_protein}")

    plt.figure(figsize=(10, 6))
    plt.scatter(F[:, 0], -F[:, 1], c='red', label='Pareto Front')
    plt.xlabel('Total Cost (Baht)')
    plt.ylabel('Total Protein (g)')
    plt.title('Pareto Front: Cost vs. Protein')
    plt.legend()
    plt.grid(True)
    plt.show()

# Selected food items for each solution
    X = res.X
    total_cost = 0
    for i in range(len(X)):
        selected_items = data[X[i].astype(bool)]

        total_cost = selected_items['Price(baht)'].sum()
        total_protein = selected_items['Protein(g)'].sum()
        total_fat = selected_items['Fat(g)'].sum()
        total_sugar = selected_items['Sugar(g)'].sum()
        total_sodium = selected_items['Sodium(mg)'].sum()
        total_calories = selected_items['Calories(kCal)'].sum()

        print(f"Meal Plan {i + 1}:")
        print(selected_items[['ITEM', 'Price(baht)', 'Calories(kCal)', 'Protein(g)', 'Fat(g)', 'Sugar(g)', 'Sodium(mg)']])
        print(f"Total cost: {total_cost}")


         # Nutritional Comparison Table
        comparison_table = [
            ["Calories (kcal)", f"{total_calories:.2f}", f"{min_cal:.0f}–{max_cal:.0f}"],
            ["Protein (g)", f"{total_protein:.2f}", f"≥ {Pconst:.0f}"],
            ["Fat (g)", f"{total_fat:.2f}", f"≤ {Fconst:.0f}"],
            ["Sugar (g)", f"{total_sugar:.2f}", f"≤ {Sconst:.0f}"],
            ["Sodium (mg)", f"{total_sodium:.2f}", f"≤ {Sodium_const:.0f}"],
        ]

        print("\n Nutritional Comparison:")
        print(tabulate(comparison_table, headers=["Nutrient", "Total in Plan", "Constraint"], tablefmt="pretty"))
        print("-" * 50)


## Plotting

In [ ]:

plt.figure(figsize=(10, 6))
plt.plot(range(1, n_runs + 1), best_costs, marker='o', label='Best Cost')
plt.plot(range(1, n_runs + 1), avg_costs, marker='s', label='Average Cost')
plt.title("Best vs Average Cost per Run")
plt.xlabel("Run")
plt.ylabel("Cost (Baht)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, n_runs + 1), best_proteins, marker='o', label='Best Protein')
plt.plot(range(1, n_runs + 1), avg_proteins, marker='s', label='Average Protein')
plt.title("Best vs Average Protein per Run")
plt.xlabel("Run")
plt.ylabel("Protein (g)")
plt.legend()
plt.grid(True)
plt.show()